<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AI_agents_versus_Agentic_AI_Claude4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OOP

In [2]:
import time

class TemperatureAgent:
    def __init__(self, target_temp, tolerance=1.0):
        self.target_temp = target_temp
        self.tolerance = tolerance
        print(f"Temperature Agent initialized with target: {target_temp}°C")

    def perceive_environment(self, current_temp):
        """Perceives the current temperature."""
        print(f"Agent perceives current temperature: {current_temp}°C")
        return current_temp

    def decide_action(self, current_temp):
        """Decides whether to turn heater on/off based on target."""
        if current_temp < self.target_temp - self.tolerance:
            return "TURN_HEATER_ON"
        elif current_temp > self.target_temp + self.tolerance:
            return "TURN_HEATER_OFF"
        else:
            return "MAINTAIN_TEMP"

    def execute_action(self, action):
        """Executes the decided action."""
        if action == "TURN_HEATER_ON":
            print("Action: Turning heater ON.")
            # In a real system, this would interact with hardware
            return "Heater is now ON."
        elif action == "TURN_HEATER_OFF":
            print("Action: Turning heater OFF.")
            # In a real system, this would interact with hardware
            return "Heater is now OFF."
        else:
            print("Action: Maintaining temperature (no change).")
            return "Temperature is stable."

    def run(self, initial_temp):
        current_temp = initial_temp
        print("--- Starting Temperature Agent Loop ---")
        for _ in range(5): # Simulate a few cycles
            perceived_temp = self.perceive_environment(current_temp)
            action = self.decide_action(perceived_temp)
            result = self.execute_action(action)
            print(f"Agent Status: {result}\n")

            # Simulate temperature change for next cycle
            if action == "TURN_HEATER_ON":
                current_temp += 1.5 # Temperature rises
            elif action == "TURN_HEATER_OFF":
                current_temp -= 1.0 # Temperature drops
            # If maintaining, assume slight fluctuation
            else:
                current_temp += 0.2
            time.sleep(0.5) # Simulate time passing
        print("--- Temperature Agent Loop Finished ---")

# --- Simulate the agent ---
if __name__ == "__main__":
    my_thermostat = TemperatureAgent(target_temp=22.0)
    my_thermostat.run(initial_temp=19.0)

Temperature Agent initialized with target: 22.0°C
--- Starting Temperature Agent Loop ---
Agent perceives current temperature: 19.0°C
Action: Turning heater ON.
Agent Status: Heater is now ON.

Agent perceives current temperature: 20.5°C
Action: Turning heater ON.
Agent Status: Heater is now ON.

Agent perceives current temperature: 22.0°C
Action: Maintaining temperature (no change).
Agent Status: Temperature is stable.

Agent perceives current temperature: 22.2°C
Action: Maintaining temperature (no change).
Agent Status: Temperature is stable.

Agent perceives current temperature: 22.4°C
Action: Maintaining temperature (no change).
Agent Status: Temperature is stable.

--- Temperature Agent Loop Finished ---


## CREWAI

In [ ]:
!pip install crewai -q # Install CrewAI
!pip install crewai-tools -q # Install CrewAI Tools
!pip install 'crewai[tools]' -q

!pip install langchain-anthropic -q

!pip install colab-env -q

In [2]:
import os
import logging
from typing import Type
from pydantic import BaseModel, Field
import colab_env

In [3]:
try:
    import litellm
    litellm.verbose = False
    logging.getLogger('litellm').setLevel(logging.CRITICAL)
except ImportError:
    pass
except Exception as e:
    print(f"Warning: Could not configure LiteLLM logging suppression due to: {e}")
    print("Consider setting environment variable LITELLM_LOG=CRITICAL to suppress all LiteLLM logs.")

from crewai import Agent, Task, Crew, Process, CrewOutput
from crewai.tools import BaseTool
from langchain_anthropic import ChatAnthropic

#llm_anthropic = ChatAnthropic(model='claude-3-7-sonnet-20250219', temperature=0.7)
#model="claude-opus-4-20250514"
model_name = 'claude-opus-4-20250514'
print('\n')
print(f"Using LLM model: {model_name}")
print('\n')
llm_anthropic = ChatAnthropic(model=model_name, temperature=0.7)

class WeatherToolInput(BaseModel):
    city: str = Field(description="The name of the city for which to check weather (e.g., 'Montreal', 'Paris').")

class WeatherTool(BaseTool):
    name: str = "Weather Checker"
    description: str = "Checks current weather conditions for a given city. Input should be the city name."
    args_schema: Type[BaseModel] = WeatherToolInput

    def _run(self, city: str) -> str:
        # Removed the print statement inside the tool's _run method
        # print(f"\n--- Calling WeatherTool for city: {city} ---")
        current_date_time = "May 22, 2025 4:34 PM EDT"

        if "montreal" in city.lower():
            return f"Weather in Montreal on {current_date_time}: Temperature 15C, Light rain, Winds 10 knots from West. Visibility 10km. IFR conditions possible but manageable."
        elif "paris" in city.lower():
            return f"Weather in Paris on {current_date_time}: Temperature 20C, Clear skies, Winds 5 knots from North. Visibility 20km. VFR conditions."
        else:
            return f"Weather data for {city} not available or city not recognized by this tool."

class AirspaceToolInput(BaseModel):
    region: str = Field(description="The name of the region for which to check airspace information (e.g., 'North Atlantic', 'Europe').")

class AirspaceTool(BaseTool):
    name: str = "Airspace Information"
    description: str = "Provides basic airspace status (e.g., restrictions, congestion) for a given region. Input should be the region name."
    args_schema: Type[BaseModel] = AirspaceToolInput

    def _run(self, region: str) -> str:
        # Removed the print statement inside the tool's _run method
        # print(f"\n--- Calling AirspaceTool for region: {region} ---")
        if "atlantic" in region.lower() or "oceanic" in region.lower():
            return "North Atlantic Oceanic airspace: No significant congestion or major restrictions reported. Few active military exercises in designated areas."
        elif "europe" in region.lower():
            return "European airspace: High traffic levels, minor delays possible around major hubs (e.g., London, Frankfurt, Paris). Some temporary flight restrictions (TFRs) for local events."
        else:
            return f"Airspace information for {region} not available or region not recognized by this tool."

weather_checker = WeatherTool()
airspace_info = AirspaceTool()

weather_analyst = Agent(
    role='Aviation Weather Analyst',
    goal='Analyze weather conditions for departure, arrival, and en-route segments to assess flight feasibility.',
    backstory=(
        "You are an expert in aviation meteorology, specializing in flight operations."
        "You understand how weather impacts aircraft and flight safety, and can interpret aviation weather reports."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[weather_checker],
    llm=llm_anthropic
)

route_planner = Agent(
    role='Aviation Route Planner',
    goal='Propose a basic flight route and assess airspace considerations based on weather and destination.',
    backstory=(
        "You are a skilled flight operations specialist with deep knowledge of air routes, "
        "navigational aids, and international airspace regulations. You aim for efficiency and safety."
        "You can devise logical flight paths considering geographical constraints and typical routes."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[airspace_info],
    llm=llm_anthropic
)

feasibility_assessor = Agent(
    role='Senior Flight Operations Manager',
    goal='Evaluate the overall feasibility of a flight based on weather analysis and proposed route, providing a final recommendation.',
    backstory=(
        "You are a highly experienced flight operations manager. Your expertise lies in synthesizing "
        "information from various sources to make critical decisions about flight safety and operational viability."
        "You make final calls on flight readiness based on all available data."
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm_anthropic
)

analyze_weather_task = Task(
    description=(
        "Analyze the current weather conditions for a flight from Montreal to Paris on May 22, 2025."
        "Focus on departure conditions in Montreal, arrival conditions in Paris,"
        "and potential en-route weather across the Atlantic. Use the 'Weather Checker' tool."
        "Summarize the key weather challenges or favorable conditions specifically impacting a transatlantic flight."
    ),
    expected_output="A concise report on weather conditions for departure, arrival, and en-route segments of a Montreal to Paris flight on May 22, 2025, highlighting any specific aviation-relevant challenges (e.g., storms, strong headwinds, low visibility) or favorable conditions.",
    agent=weather_analyst,
    tools=[weather_checker]
)

plan_route_task = Task(
    description=(
        "Given the flight path from Montreal to Paris, propose a basic flight route."
        "Assess any known restrictions or congestion in the North Atlantic Oceanic and European airspace. Use the 'Airspace Information' tool."
        "Outline the proposed route segments and any potential airspace-related challenges or recommended considerations for the pilot."
    ),
    expected_output="A basic transatlantic flight route proposal from Montreal to Paris, including general segments and a summary of any identified airspace considerations or potential traffic delays.",
    agent=route_planner,
    tools=[airspace_info],
    context=[analyze_weather_task]
)

assess_feasibility_task = Task(
    description=(
        "Evaluate the overall feasibility of the flight from Montreal to Paris on May 22, 2025,"
        "based on the weather analysis and the proposed route plan provided by the other agents."
        "Consider the combined impact of weather (from Weather Analyst) and airspace information (from Route Planner)."
        "Provide a final, clear recommendation on whether the flight should 'PROCEED', 'DELAY', or 'CANCEL',"
        "along with a concise justification based on the synthesized information."
    ),
    expected_output="A final flight feasibility assessment and recommendation (PROCEED, DELAY, or CANCEL) for the Montreal to Paris flight on May 22, 2025, with a brief, clear justification. Example: 'RECOMMENDATION: PROCEED. Justification: Favorable weather, clear airspace.' or 'RECOMMENDATION: DELAY. Justification: Strong headwinds en-route and potential arrival congestion.'",
    agent=feasibility_assessor,
    context=[analyze_weather_task, plan_route_task]
)

flight_crew = Crew(
    agents=[weather_analyst, route_planner, feasibility_assessor],
    tasks=[analyze_weather_task, plan_route_task, assess_feasibility_task],
    process=Process.sequential,
    verbose=False # Set verbose to False to avoid the recursion error
)

print("## Running the Flight Feasibility Crew")
result = flight_crew.kickoff()

print("\n## Crew Run Complete!")
print("## Final Recommendation:")
print(result)



Using LLM model: claude-opus-4-20250514


## Running the Flight Feasibility Crew
# Agent: Aviation Weather Analyst
## Task: Analyze the current weather conditions for a flight from Montreal to Paris on May 22, 2025.Focus on departure conditions in Montreal, arrival conditions in Paris,and potential en-route weather across the Atlantic. Use the 'Weather Checker' tool.Summarize the key weather challenges or favorable conditions specifically impacting a transatlantic flight.


# Agent: Aviation Weather Analyst
## Thought: I need to analyze the weather conditions for a transatlantic flight from Montreal to Paris. I'll check the current weather in both cities to assess departure and arrival conditions. Since I cannot directly check en-route weather over the Atlantic with the available tool, I'll focus on the departure and arrival airports and infer potential en-route conditions based on the weather patterns at both ends.
Let me start by checking the weather in Montreal for departure condi